In [1]:
import os
import numpy as np
import tensorflow as tf

from skimage import exposure
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.layers import Input, Dense, add, Activation, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, BatchNormalization
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib

## Use GPU

In [2]:
# config = tf.compat.v1.ConfigProto(config=tf.ConfigProto(log_device_placement=True))
# sess = tf.compat.v1.Session(config=config) 
# K.set_session(sess)

# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
# print(device_lib.list_local_devices())
# K.tensorflow_backend._get_available_gpus()

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18057891840933340044]

## Download dataset 
https://drive.google.com/drive/u/3/folders/1sHh6NvuKX6RB5OytLwf4kaqfQ9svJNDQ

## Load data

In [3]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

print(x_train.shape, 'train samples')
print(x_test.shape[0], 'test samples')

(50000, 32, 32, 3) train samples
10000 test samples


In [4]:
# It's a multi-class classification problem 
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6,'horse': 7,'ship': 8, 'truck': 9}
print(np.unique(y_train))

[0 1 2 3 4 5 6 7 8 9]


![image](https://img-blog.csdnimg.cn/20190623084800880.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lqcDE5ODcxMDEz,size_16,color_FFFFFF,t_70)

## Data preprocess

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to one-hot encoding (keras model requires one-hot label as inputs)
num_classes = 10
print(y_train[0])
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[0])

[9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


## Data Augmentation

In [6]:
x_train.shape

(50000, 32, 32, 3)

In [7]:
def AHE(img):
    img_adapteq = exposure.equalize_adapthist(img, clip_limit=0.03)
    return img_adapteq

# train_datagen = ImageDataGenerator()
train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=0.125,
    height_shift_range=0.125,
    fill_mode='constant',
    cval=0.,
    rotation_range=15)
#     preprocessing_function=AHE,
#     rescale=1,
#     rotation_range=15,
#     zoom_range=0.1,
#     horizontal_flip=True,
#     vertical_flip=True,
#     fill_mode='nearest')
    
# train_datagen.fit(x_train)

## Early Stopping

In [8]:
keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

## Build model

In [9]:
def residual_block(x,o_filters,increase=False):
        stride = (1,1)
        if increase:
            stride = (2,2)

        o1 = Activation('relu')(BatchNormalization(momentum=0.9, epsilon=1e-5)(x))
        conv_1 = Conv2D(o_filters,kernel_size=(3,3),strides=stride,padding='same',
                        kernel_initializer="he_normal",
                        kernel_regularizer=regularizers.l2(weight_decay))(o1)
        o2  = Activation('relu')(BatchNormalization(momentum=0.9, epsilon=1e-5)(conv_1))
        conv_2 = Conv2D(o_filters,kernel_size=(3,3),strides=(1,1),padding='same',
                        kernel_initializer="he_normal",
                        kernel_regularizer=regularizers.l2(weight_decay))(o2)
        if increase:
            projection = Conv2D(o_filters,kernel_size=(1,1),strides=(2,2),padding='same',
                                kernel_initializer="he_normal",
                                kernel_regularizer=regularizers.l2(weight_decay))(o1)
            block = add([conv_2, projection])
        else:
            block = add([conv_2, x])
        return block

In [10]:
# # Builde model
# model = Sequential() # Sequential groups a linear stack of layers 
# model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=x_train.shape[1:])) # Add Convolution layers
# model.add(Activation('relu')) # Add Relu activation for non-linearity
# model.add(Conv2D(filters=32, kernel_size=(3, 3))) # Add Convolution layers
# model.add(Activation('relu')) # Add Relu activation for non-linearity
# model.add(MaxPooling2D(pool_size=(4, 4))) # Add Max pooling to lower the sptail dimension

# model.add(Flatten()) # Flatten the featuremaps
# model.add(Dense(units=512)) # Add dense layer with 512 neurons
# model.add(Activation('relu')) # Add Relu activation for non-linearity
# model.add(Dense(units=num_classes)) # Add final output layer for 10 classes
# model.add(Activation('softmax')) # Add softmax activation to transfer logits into probabilities

# =========

# model = Sequential()
# model.add(Conv2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
# model.add(Conv2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=2))

# model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
# model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=2))

# model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
# model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=2))

# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(rate=0.25))
# model.add(Dense(10, activation='softmax'))

# =========

# model = Sequential()

# model.add(Conv2D(32, kernel_size=(3, 3), input_shape=x_train.shape[1:], activation ="relu"))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.3))

# model.add(Conv2D(64, kernel_size=(3, 3), input_shape=x_train.shape[1:], activation ="relu"))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.3))

# model.add(Conv2D(128, kernel_size=(3, 3), input_shape=x_train.shape[1:], activation ="relu"))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.4))

# model.add(Flatten())
# model.add(Dense(80, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(num_classes, activation='softmax'))

# =========

stack_n            = 1
layers             = 6 * stack_n + 2
num_classes        = 10
img_rows, img_cols = 32, 32
img_channels       = 3
batch_size         = 128
epochs             = 200
iterations         = 50000 // batch_size + 1
weight_decay       = 1e-4
classes_num        = 10

img_input = Input(shape=(x_train.shape[1:]))

x = Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),padding='same',
               kernel_initializer="he_normal",
               kernel_regularizer=regularizers.l2(weight_decay))(img_input)

# input: 32x32x16 output: 32x32x16
for _ in range(stack_n):
    x = residual_block(x,16,False)

# input: 32x32x16 output: 16x16x32
x = residual_block(x,32,True)
for _ in range(1,stack_n):
    x = residual_block(x,32,False)

# input: 16x16x32 output: 8x8x64
x = residual_block(x,64,True)
for _ in range(1,stack_n):
    x = residual_block(x,64,False)

x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
x = Activation('relu')(x)
x = GlobalAveragePooling2D()(x)

# input: 64 output: 10
x = Dense(classes_num,activation='softmax',kernel_initializer="he_normal",
          kernel_regularizer=regularizers.l2(weight_decay))(x)

model = Model(img_input, x)

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

## Set Optimizer

In [12]:
opt = keras.optimizers.SGD(learning_rate=0.005, momentum=0.9, nesterov=True)

## Training

In [13]:
# Compile the model with loss function and optimizer, and evaluate with accuracy
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Setup some hyperparameters
# batch_size = 32
# epochs = 100

# Fit the data into model
# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_data=(x_test, y_test),
#           shuffle=True)

model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
    epochs=epochs,
    steps_per_epoch=iterations,
    validation_data=(x_test, y_test))

/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
391/391 [==============================] - 86s 219ms/step - loss: 1.9845 - accuracy: 0.2930 - val_loss: 1.8220 - val_accuracy: 0.3599
Epoch 2/200
391/391 [==============================] - 86s 219ms/step - loss: 1.7165 - accuracy: 0.3847 - val_loss: 1.6224 - val_accuracy: 0.4148
Epoch 3/200
391/391 [==============================] - 86s 219ms/step - loss: 1.5956 - accuracy: 0.4368 - val_loss: 1.5078 - val_accuracy: 0.4729
Epoch 4/200
391/391 [==============================] - 86s 219ms/step - loss: 1.4995 - accuracy: 0.4779 - val_loss: 1.4498 - val_accuracy: 0.4926
Epoch 5/200
391/391 [==============================] - 86s 219ms/step - loss: 1.4247 - accuracy: 0.5113 - val_loss: 1.4468 - val_accuracy: 0.5082
Epoch 6/200
391/391 [==============================] - 86s 219ms/step - loss: 1.3650 - accuracy: 0.5340 - val_loss: 1.3008 - val_accuracy: 0.5566
Epoch 7/200
391/391 [==============================] - 86s 219ms/step - loss: 1.3147 - accuracy: 0.5528 - val_loss: 1.2929 -

In [14]:
y_pred = model.predict(x_test)
print(y_pred.shape) # 10000 samples, each sample with probaility of 10 classes

(10000, 10)


In [15]:
y_pred[0] 

array([2.81414742e-07, 9.94904757e-01, 1.16518947e-10, 1.09749344e-07,
       5.31831038e-11, 1.43482684e-10, 1.13031329e-12, 7.58477459e-09,
       9.55867776e-08, 5.09477640e-03], dtype=float32)

In [16]:
np.argmax(y_pred[0]) # argmax to find the predict class with highest probability. 9=truck

1

In [17]:
y_pred = np.argmax(y_pred, axis=1)

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [18]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

In [19]:
assert y_pred.shape == (10000, )

In [20]:
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test set:  0.8096
